In [13]:
import csv
import math
def areawise(freqfile,percentfile):
    csvfile = open(percentfile, newline='')
    csvAvg = open(freqfile,newline='')
    
    c = csv.reader(csvfile)
    d = csv.reader(csvAvg)
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    average=[0] * 71  # change to higher value to make it non-redundant for now it is equal to the no of total unique crimes
    i=0
    yearsToTrain = 14    # change s value to no of years to be used for training 
    
    for row in d:
        if row[0] == 'TOTAL'or row[0]=='Category' or row[0]=='year' or row[0]=='month':
            continue
        else:
           j=0
           for month in range (1,(yearsToTrain*12)+1):
               if float(row[month]) !=0:    
                   j+=1
                   average[i] += float(row[month])    # maintain grand average for scale later 
           if average[i] != 0:
                average[i] /= j
           i+=1
          
    percentages = []
    totals = []
    categories =[]
    i = 0
    monYear = []
    
    for row in c:
        
        if row[0] == 'TOTAL'or row[0]=='Category' or row[0]=='year':
            continue
        elif row[0]=='month':
            for month in range(1,(yearsToTrain*12)+1):
                monYear.append(float(row[month]))
        else:
           temp = []
           categories.append(row[0])
           
           for month in range (1,(yearsToTrain*12)+1):
               if float(row[month]) !=0:    
                   j+=1
               
               temp.append(float(row[month]))   # add monthly percentages of a particular crime to  list 
           
           percentages.append(temp)   # add monthly percentages list to a list 
           i += 1
    
    predictions=[]


    def predictOverYears(start,end):           # fn to give prediction for one year mentioned by start and end
        predictAyear = []
        for idx in range (0,i):
            crime = percentages[idx]
            sum =0
            index = -1
            for it in range (start,end):     # sum for caluclationg confidence later on             
                sum+=crime[it]
        
            predictMonth = []
    
            for it in range (start,end):
                avg = sum - crime[it]
                avg=float(avg/11)
                if avg != 0:
                    likeness = ((float(crime[it])/(avg))) * 100    # measure confidence rate for that month for that crime
                else :
                    likeness=0
                temp = [likeness,monYear[it]]
                predictMonth.append(temp)                   # append confidence rate and index of month predicted
    
            predictAyear.append(predictMonth)           # append all month's of that year likliness to a list 
        return predictAyear


    for j in range(0,yearsToTrain):            
        n=j*12
        predictions.append(predictOverYears(n,n+12))
    
    # print(predictions[1])
    
    predictionPerCrime = []
    for idx in range(0,i):
        tem = []
        for k in range(0,yearsToTrain):
            tem.append(predictions[k][idx])           # get each individual year predictions and add it to a new list 
        predictionPerCrime.append(tem)
        
    # print(predictionPerCrime[0])



    grandPrediction = []
    for idx in range(0,i):
        perCrimeConf=[]
        for month in range(0,12):
            sumOverYears=0
            for yr in range(0,yearsToTrain):
                sumOverYears+=predictionPerCrime[idx][yr][month][0]
            tem=sumOverYears/yearsToTrain   # sum of confidence of a month across years and divide by yearsToTrain
            perCrimeConf.append(tem)
        grandPrediction.append(perCrimeConf)  # append confidence rates of each month per crime to list 

    
    
    # print(grandPrediction[0][2])
    
    for j in range(0,i):
        low=9999
        for m in range(0,12):
            if grandPrediction[j][m] < low:
                low = grandPrediction[j][m]
        if low != 0 :
             low = math.ceil(low) -2  # find lowest value ceil it and subtrat 2 from it
        for n in range(0,12):
            grandPrediction[j][n] -= low  # then subtract this val(low) from all
    
    print("{")
    print('\"'+freqfile.split('_')[0]+'\":')
    print("[")
    


    for j in range(0,i):
        print('{')
        print('\"'+"crime"+'\":'+'\"'+categories[j]+'\",')
        max = 0
        avg = 0
        for m in range (0,12):
             avg+=grandPrediction[j][m]
             if grandPrediction[j][m] > max :
                 max = grandPrediction[j][m]
        avg=avg/12
        ideal = ( max + avg)/2     # compute ideal confidence and acceptable confidence level
        print('\"'+"overallAverage"+'\":'+'\"'+str(average[j])+'\",')
        print('\"'+"idealConfidence"+'\":'+'\"'+str(ideal)+'\",')
        print('\"'+"accepatableConfidence"+'\":'+'\"'+str(avg)+'\",')
        print('\"'+"confidenceRates"+'\":'+'[')
        print('{')
        for k in range(0,12):
            if k != 11:
                print('\"'+months[k]+'\":'+'\"'+str(grandPrediction[j][k])+'\",')
            else:
                print('\"'+months[k]+'\":'+'\"'+str(grandPrediction[j][k])+'\"')
        # print(j+1,")",categories[j],"With overall average:",average[j])
        # print("With ideal confidence:",ideal,"and acceptable confidence:",avg,"monthly predictons are:")
        # for k in range(0,12):
        #     print("     Month",k+1,"confidence rate:",grandPrediction[j][k]) 
        print('}')  
        print(']')
        if j!=i-1:
            print('},')
        else:
            print('}')
                 
    print(']')
    print('}')

areawise('BRONX_Frequency.csv','BRONX_PercentageTotal.csv')
areawise('BROOKLYN_Frequency.csv','BROOKLYN_PercentageTotal.csv')
areawise('MANHATTAN_Frequency.csv','MANHATTAN_PercentageTotal.csv')
areawise('QUEENS_Frequency.csv','QUEENS_PercentageTotal.csv')
areawise('STATEN_ISLAND_Frequency.csv','STATEN_ISLAND_PercentageTotal.csv')



{
"BRONX":
[
{
"crime":"HARRASSMENT 2",
"overallAverage":"1145.9345238095239",
"idealConfidence":"8.895779490453231",
"accepatableConfidence":"7.025592118516644",
"confidenceRates":[
{
"Jan":"5.17650569023499",
"Feb":"3.4070378063325535",
"Mar":"6.598562484788545",
"Apr":"7.890241082144712",
"May":"10.35270529562375",
"Jun":"10.460208194904226",
"Jul":"6.041754179988814",
"Aug":"1.6215842509570848",
"Sep":"10.765966862389817",
"Oct":"8.841792733366319",
"Nov":"5.97855670383926",
"Dec":"7.172190137629656"
}
]
},
{
"crime":"PETIT LARCENY",
"overallAverage":"1191.125",
"idealConfidence":"14.222823200892229",
"accepatableConfidence":"11.039392645528805",
"confidenceRates":[
{
"Jan":"9.096441460113525",
"Feb":"1.8208860853705175",
"Mar":"3.365134258098095",
"Apr":"6.897539827386964",
"May":"10.045222945585792",
"Jun":"12.900148112177334",
"Jul":"12.589187355251838",
"Aug":"14.766908410072531",
"Sep":"15.164436626013895",
"Oct":"14.693462686380599",
"Nov":"13.72709022363891",
"Dec":"17.40625

In [1]:
test = 'whatter_what'
test.split('_')[0]
freqfile='BROOKLYN_Frequency.csv'
print('\"'+freqfile.split('_')[0]+'\":')

print("{")
print('\"'+freqfile.split('_')[0]+'\":')
print("[")

areawise('BRONX_Frequency.csv','BRONX_PercentageTotal.csv')
areawise('BROOKLYN_Frequency.csv','BROOKLYN_PercentageTotal.csv')
areawise('MANHATTAN_Frequency.csv','MANHATTAN_PercentageTotal.csv')
areawise('QUEENS_Frequency.csv','QUEENS_PercentageTotal.csv')
areawise('STATEN_ISLAND_Frequency.csv','STATEN_ISLAND_PercentageTotal.csv')


"BROOKLYN":
